Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1112]:
from itertools import product
from functools import reduce
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse
from copy import copy

In [1113]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [1114]:
PROBLEM_SIZE = 100
NUM_SETS = PROBLEM_SIZE
CHANCE = .3
SETS = make_set_covering_problem(PROBLEM_SIZE, NUM_SETS, CHANCE).toarray()
print("Element at row=42 and column=42:", SETS[42, 42])

Element at row=42 and column=42: True


In [1115]:
def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i, t in enumerate(state) if t],
        np.array([False for _ in range(PROBLEM_SIZE)]),
    )

def goal_check(state):
    return np.all(covered(state))

In [1116]:
def fitness(state):
    cost = sum(state)
    valid = np.sum(covered(state))
    return valid, -cost

def tweak1(state):
    new_state = copy(state)
    index = randint(0, PROBLEM_SIZE - 1)
    new_state[index] = not new_state[index]
    return new_state

tweak = tweak1

In [1117]:
def random_mutation_HC(state):
    step = 0
    while not goal_check(state) and step < 100_000:
        step += 1
        new_state = tweak(copy(state))
        if fitness(new_state) > fitness(state):
            state = new_state
    return state

def steepest_step_HC(state):
    step = 0
    while not goal_check(state) and step < 10_000:
        step += 1
        new_state = tweak(copy(state))
        for _ in range(10):
            new_new_state = tweak(copy(state))
            if fitness(new_new_state) > fitness(new_state):
                new_state = new_new_state
        if fitness(new_state) > fitness(state):
            state = new_state
    return state

def simulated_annealing(state):
    current_sol = fitness(state)
    best_sol = current_sol
    t = 1000
    while t > 0.1:
        step = 0
        while step < 100:
            step += 1
            new_state = tweak(copy(state))
            new_sol = fitness(new_state)
            if new_sol > current_sol or random() < np.exp(-(current_sol[1]-new_sol[1])/t):
                state = new_state
                current_sol = new_sol
                if current_sol > best_sol:
                    best_sol = current_sol
        t *= 0.995
    return state, best_sol

In [1118]:
current_state = [False for _ in range(NUM_SETS)]
best_sol = (0, -PROBLEM_SIZE)
tot_sets = 0
num_sol = 0

for _ in range(1):
    current_state = random_mutation_HC(current_state)
    current_sol = fitness(current_state)
    if current_sol[0]:
        tot_sets += current_sol[1]
        num_sol += 1
    if current_sol > best_sol:
        best_sol = current_sol
print("Best solution:", best_sol)

Best solution: (100, -11)


In [1119]:
current_state = [False for _ in range(NUM_SETS)]
best_sol = (0, -PROBLEM_SIZE)
tot_sets = 0
num_sol = 0

for _ in range(1):
    current_state = steepest_step_HC(current_state)
    current_sol = fitness(current_state)
    if current_sol[0]:
        tot_sets += current_sol[1]
        num_sol += 1
    if current_sol > best_sol:
        best_sol = current_sol
print("Best solution:", best_sol)

Best solution: (100, -8)


In [1120]:
current_state = [choice([True, False]) for _ in range(NUM_SETS)]
best_sol = (0, -PROBLEM_SIZE)
tot_sets = 0
num_sol = 0

for _ in range(1):
    current_state, best_sol = simulated_annealing(current_state)
    current_sol = fitness(current_state)
    if current_sol[0]:
        tot_sets += current_sol[1]
        num_sol += 1
print("Best solution:", best_sol)

Best solution: (100, -8)
